In [1]:
import pymssql as ms_sql
import pymysql as mysql
import datetime
import progressbar

In [2]:
def connect_mssql():
    ms_sql_base = 'voice_ai'
    ms_sql_server = '10.2.4.124'
    ms_sql_login = 'ICECORP\\1c_sql'

    with open('sql.pass', 'r') as file:
        ms_sql_pass = file.read().replace('\n', '')
        file.close()

    return ms_sql.connect(ms_sql_server,
                          ms_sql_login,
                          ms_sql_pass,
                          ms_sql_base)


def connect_mysql():

    mysql_name = 'MICO_96'
    mysql_server = '10.2.4.146'
    mysql_login = 'asterisk'
    with open('mysql.pass','r') as file:
        mysql_pass = file.read().replace('\n', '')
        file.close()
    #mysql_pass = '12WsxcdE34'
    
    return mysql.connect(
        mysql_server, 
        mysql_login, 
        mysql_pass,
        mysql_name
    )

### source_id

In [24]:
ms_sql_conn = connect_mssql()
job_len = 0
with ms_sql_conn:
    print('queue')
    query = "select count(filename) from queue where source_id is NULL;"
    cursor = ms_sql_conn.cursor()
    cursor.execute(query)
    for row in cursor.fetchall():
        print([r for r in row])
        
    query = "select count(filename) from queue where source_id is not NULL;"
    cursor = ms_sql_conn.cursor()
    cursor.execute(query)
    for row in cursor.fetchall():
        print([r for r in row])
    
    print('transcribations')
    query = "select count(distinct audio_file_name) from transcribations where source_id is NULL;"
    cursor = ms_sql_conn.cursor()
    cursor.execute(query)
    for row in cursor.fetchall():
        print([r for r in row])
        
    query = "select count(distinct audio_file_name) from transcribations where source_id is not NULL;"
    cursor = ms_sql_conn.cursor()
    cursor.execute(query)
    for row in cursor.fetchall():
        print([r for r in row])
        
    """query = "select source_id from transcribations where source_id is not NULL;"
    cursor = ms_sql_conn.cursor()
    cursor.execute(query)
    for row in cursor.fetchall():
        print([r for r in row])"""

queue
[0]
[370]
transcribations
[0]
[267759]


In [23]:
"""# update
ms_sql_conn = connect_mssql()
job_len = 0
with ms_sql_conn:
    print('queue')
    #query = "select count(filename) from queue where source_id is NULL;"
    #query = "update queue set source_id = 1 where source_id is NULL;"
    query = "update transcribations set source_id = 1 where source_id is NULL;"
    cursor = ms_sql_conn.cursor()
    cursor.execute(query)
    #for row in cursor.fetchall():
    #    print([r for r in row])
    ms_sql_conn.commit()"""

queue


### queue

In [3]:
ms_sql_conn = connect_mssql()
job_len = 0
with ms_sql_conn:
    query = "select top 1 audio_file_name from transcribations;"
    cursor = ms_sql_conn.cursor()
    cursor.execute(query)
    for row in cursor.fetchall():
        print([r for r in row])

['in_4957237230_2020-10-09-07-36-05rxtx.wav']


In [4]:
cpu_cores = [i for i in range(0,15)]

In [7]:
#cursor = self.conn.cursor()
ms_sql_conn = connect_mssql()
with ms_sql_conn:
    cursor = ms_sql_conn.cursor()
    sql_query = '''
    IF OBJECT_ID('tempdb..#tmp_cpu_queue_len') IS NOT NULL
    DROP TABLE #tmp_cpu_queue_len;

    CREATE TABLE #tmp_cpu_queue_len
    (
    cpu_id INT,
    files_count int
    );

    INSERT INTO #tmp_cpu_queue_len 
    '''
    for i in cpu_cores:
        if i==0:
            sql_query += 'select 0 as cpu_id, 0 as files_count '
        else:
            sql_query += 'union all select '+str(i)+',0 '
    sql_query += 'union all	select cpu_id, count(filename) from queue group by cpu_id; '
    sql_query += 'select top 1 cpu_id, max(files_count) '
    sql_query += 'from #tmp_cpu_queue_len group by cpu_id order by max(files_count), cpu_id;'
    
    cursor.execute(sql_query)
    result = 0
    for row in cursor.fetchall():
        result+=1
        cpu_id = int(row[0])
    if result==0:
        print('error: unable to get shortest_queue_cpu')
        cpu_id = 0

In [8]:
result, cpu_id

(1, 10)

### regexp date from file

In [33]:
#bar = progressbar.ProgressBar(maxval=job_len).start()
ms_sql_conn = connect_mssql()
job_len = 0
with ms_sql_conn:
    query = "select top 2 \
    CAST(CAST(date_y AS varchar) + '-' + CAST(date_m AS varchar) + '-' + CAST(date_d AS varchar) AS DATETIME) \
    from transcribations;"
    query = "select top 1 audio_file_name from transcribations;"
    
    query = "select count(id) from transcribations where record_date is Null;"
    query = "update transcribations set record_date = \
    CAST(CAST(date_y AS varchar) + '-' + CAST(date_m AS varchar) + '-' + CAST(date_d AS varchar) AS DATETIME) \
    where record_date is Null_;"
    
    query = "select top 2 \
    CAST(CAST(date_y AS varchar) + '-' + CAST(date_m AS varchar) + '-' + CAST(date_d AS varchar) AS DATETIME) \
    from transcribations \
    where record_date is Null;"
    #'%[^-A-Za-z0-9/.+$]%'
    
    query = "select top 2 \
    audio_file_name, \
    PATINDEX('%____-__-__-__-__-__%', audio_file_name), \
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) , 4 ), \
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name)+5 , 2 ), \
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name)+8 , 2 ), \
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name)+11 , 2 ), \
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name)+14 , 2 ), \
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name)+17 , 2 ) \
    from transcribations \
    where record_date is Null;"
    #'%[^-A-Za-z0-9/.+$]%'
    #SUBSTRING(name, 2, 3) 
    #\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2}
    
    query = "select top 2 \
    audio_file_name, \
    CAST( \
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) , 10 ) + ' ' + \
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) +11 , 2 ) + ':' + \
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) +14 , 2 ) + ':' +\
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) +17 , 2 ) \
    AS DATETIME) \
    from transcribations \
    where record_date is Null;"
    
    query = "select top 2 \
    audio_file_name, \
    CAST( \
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) , 10 ) + ' ' + \
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) +11 , 2 ) + ':' + \
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) +14 , 2 ) + ':' +\
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) +17 , 2 ) \
    AS DATETIME) \
    from transcribations \
    where CAST( \
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) , 10 ) + ' ' + \
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) +11 , 2 ) + ':' + \
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) +14 , 2 ) + ':' +\
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) +17 , 2 ) \
    AS DATETIME) is Null;"
    
    
    
    query = "update transcribations set record_date = \
    CAST( \
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) , 10 ) + ' ' + \
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) +11 , 2 ) + ':' + \
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) +14 , 2 ) + ':' +\
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) +17 , 2 ) \
    AS DATETIME) \
    where record_date is Null;"
    
    query = "select count(id) from transcribations where record_date is Null;"
    
    cursor = ms_sql_conn.cursor()
    cursor.execute(query)
    for row in cursor.fetchall():
        print([r for r in row])

[7413999]


In [41]:
ms_sql_conn = connect_mssql()
job_len = 0
with ms_sql_conn:
    query = "select count(id) from transcribations where record_date is Null;"
    
    cursor = ms_sql_conn.cursor()
    cursor.execute(query)
    for row in cursor.fetchall():
        print([r for r in row])

[7413999]


In [64]:
ms_sql_conn = connect_mssql()
job_len = 0
with ms_sql_conn:    
    
    query = "update transcribations set record_date = \
    CAST( \
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) , 10 ) + ' ' + \
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) +11 , 2 ) + ':' + \
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) +14 , 2 ) + ':' +\
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) +17 , 2 ) \
    AS DATETIME) \
    where record_date is Null;"
    
    query = "select top 10 \
    audio_file_name, \
    CAST( \
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) , 10 ) + ' ' + \
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) +11 , 2 ) + ':' + \
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) +14 , 2 ) + ':' +\
    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) +17 , 2 ) \
    AS TEXT) \
    from transcribations \
    where record_date is Null;"
    
    
    
    
    
    #cursor = ms_sql_conn.cursor()
    #cursor.execute(query)
    #for row in cursor.fetchall():
    #    print([r for r in row])
    print(query)

update transcribations set record_date =     CAST(     SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) , 10 ) + ' ' +     SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) +11 , 2 ) + ':' +     SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) +14 , 2 ) + ':' +    SUBSTRING ( audio_file_name ,PATINDEX('%____-__-__-__-__-__%', audio_file_name) +17 , 2 )     AS DATETIME)     where record_date is Null;


In [65]:
ms_sql_conn = connect_mssql()
job_len = 0
with ms_sql_conn:
    query = "select count(id) from transcribations where record_date is Null;"
    
    cursor = ms_sql_conn.cursor()
    cursor.execute(query)
    for row in cursor.fetchall():
        print([r for r in row])

[0]


In [33]:
row[0]

'in_4957237230_2020-10-09-07-36-05rxtx.wav'

In [35]:
regex = '\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2}'

In [34]:
import re

In [39]:
rec_source_date = re.findall(r'\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2}', row[0])[0]
rec_source_date

'2020-10-09-07-36-05'

In [48]:
rec_date = rec_source_date[:10] + ' ' + rec_source_date[11:].replace('-', ':')
rec_date

'2020-10-09 07:36:05'

In [7]:
def read_job_len():
    ms_sql_conn = connect_mssql()
    # read job length
    job_len = 0
    with ms_sql_conn:
        query = """select
        count(distinct audio_file_name)
        from transcribations
        where audio_file_name!='' and
        text!='' and
        dst is null;"""        
        cursor = ms_sql_conn.cursor()
        cursor.execute(query)
        for row in cursor.fetchall():
            job_len = row[0]

    if job_len == 0:
        print('nothing to do. exit')
        exit()

    return job_len

In [4]:
def read_file_name():
    ms_sql_conn = connect_mssql()
    file_name = ''
    # read filename from transcribations
    with ms_sql_conn:
        query = """select top 1
        audio_file_name,
        date_y,
        date_m,
        date_d
        from transcribations
        where text!='' and
        dst is null and
        audio_file_name!=''
        order by
        date_y,
        date_m,
        date_d,
        transcribation_date;"""
        cursor = ms_sql_conn.cursor()
        cursor.execute(query)
        for row in cursor.fetchall():
            file_name = row[0]
            date_y = row[1]
            date_m = row[2]
            date_d = row[3]

    if file_name == '':
        print('empty filename. exit')
        exit()

    return file_name, date_y, date_m, date_d

In [14]:
def read_dst(file_name, date_y, date_m, date_d):
    #query = "select SUBSTRING(dstchannel, 5, 4) from PT1C_cdr_MICO limit 10"
    my_sql_conn = connect_mysql()
    # read linkedid from CDR
    linkedid = ''
    filename = file_name.replace('rxtx.wav', '')
    idy, idm, idd = int(date_y), int(date_m), int(date_d)
    date_from = datetime.datetime(idy, idm, idd)
    date_toto = date_from+datetime.timedelta(days=1)
    sdf = str(date_from)
    f_0 = '%Y-%m-%d %H:%M:%S'
    f_1 = '%Y-%m-%dT%H:%M:%S'
    date_from = datetime.datetime.strptime(sdf, f_0).strftime(f_1)
    date_toto = datetime.datetime.strptime(str(date_toto), f_0).strftime(f_1)
    with my_sql_conn:
        query = """select
            SUBSTRING(dstchannel, 5, 4) as dst
            from PT1C_cdr_MICO as PT1C_cdr_MICO
            where
            calldate>'"""+date_from+"""' and
            calldate<'"""+date_toto+"""' and
            PT1C_cdr_MICO.recordingfile LIKE '%"""+filename+"""%'
            limit 1"""
        cursor = my_sql_conn.cursor()
        cursor.execute(query)
        for row in cursor.fetchall():
            dst = row[0]

    if dst == '':
        print('linkedid for '+filename+' is empty. exit')
        exit()

    return dst

In [25]:
def update_transcribations(file_name, dst):
    ms_sql_conn = connect_mssql()
    with ms_sql_conn:
        cursor = ms_sql_conn.cursor()
        query = """update transcribations
        set dst = '"""+dst+"""'
        where audio_file_name='"""+file_name+"';"
        print(query)
        #cursor.execute(query)
        #ms_sql_conn.commit()

In [22]:
file_name, date_y, date_m, date_d = read_file_name()
file_name

'in_9163652085_2020-10-09-07-11-39rxtx.wav'

In [23]:
dst = read_dst(file_name, date_y, date_m, date_d)
dst

'5011'

In [26]:
update_transcribations(file_name, dst)

update transcribations
        set dst = '5011'
        where audio_file_name='in_9163652085_2020-10-09-07-11-39rxtx.wav';


In [27]:
job_len = read_job_len()

bar = progressbar.ProgressBar(maxval=job_len).start()
step = 0
#while(True):    
#file_name, date_y, date_m, date_d = read_file_name()
ms_sql_conn = connect_mssql()
file_name = ''
# read filename from transcribations
with ms_sql_conn:
    query = """select
    audio_file_name,
    date_y,
    date_m,
    date_d
    from transcribations
    where text!='' and
    dst is null and
    audio_file_name!=''
    order by
    date_y,
    date_m,
    date_d,
    transcribation_date;"""
    cursor = ms_sql_conn.cursor()
    cursor.execute(query)
    for row in cursor.fetchall():
        bar.update(step)
        file_name = row[0]
        date_y = row[1]
        date_m = row[2]
        date_d = row[3]
        dst = read_dst(file_name, date_y, date_m, date_d)
        update_transcribations(file_name, dst)
        step += 1
        break

NameError: name 'progressbar' is not defined

# monitoring

In [20]:
def queue_len():
    result = ''
    ms_sql_conn = connect_mssql()
    with ms_sql_conn:
        query = 'select count(distinct filename) as queued from queue;'
        cursor = ms_sql_conn.cursor()
        cursor.execute(query)
        for row in cursor.fetchall():
            result += 'В очереди: ' + str(row[0])
    return result
queue_len()

'В очереди: 135'

In [23]:
def queue_by_cpu():
    result = 'Очередь по ядрам:\n'
    ms_sql_conn = connect_mssql()
    with ms_sql_conn:
        query = 'select cpu_id, count(distinct filename) as filename from queue group by cpu_id order by cpu_id;'
        cursor = ms_sql_conn.cursor()
        cursor.execute(query)
        for row in cursor.fetchall():
            result += str(row[0])+': '+str(row[1])+'\n'
    return result
queue_by_cpu()

'Очередь по ядрам:0: 10\n1: 11\n2: 11\n3: 7\n4: 10\n5: 10\n6: 7\n7: 9\n8: 10\n9: 5\n10: 9\n11: 6\n12: 7\n13: 9\n14: 10\n'

In [29]:
#datetime.datetime.now()
#date_toto = date_from+datetime.timedelta(days=1)


('2021-01-31T00:00:00', '2021-02-01T00:00:00')

In [37]:
def transcribed_yesterday():
    result = 'Распознанно за вчера: '

    currentdate = datetime.datetime.today()
    start_of_day = currentdate.combine(currentdate.date(), currentdate.min.time())
    yesterday = start_of_day + datetime.timedelta(days=-1)
    df = str(yesterday)
    dt = str(start_of_day)
    f_0 = '%Y-%m-%d %H:%M:%S'
    f_1 = '%Y-%m-%dT%H:%M:%S'
    date_from = datetime.datetime.strptime(df, f_0).strftime(f_1)
    date_toto = datetime.datetime.strptime(dt, f_0).strftime(f_1)
    date_from, date_toto
    
    ms_sql_conn = connect_mssql()
    with ms_sql_conn:
        query = "select count(distinct audio_file_name) as transcribed from transcribations"
        query += " where transcribation_date>'"+date_from+"'"
        query += " and transcribation_date<'"+date_toto+"';"
        cursor = ms_sql_conn.cursor()
        cursor.execute(query)
        for row in cursor.fetchall():
            result += str(row[0])
    return result

transcribed_yesterday()

'Распознанно за вчера: 1588'